## What this does  
    Selects all columns from adss_data_part1.
    Joins with beale_decoder to replace numeric beale codes with beale_label.
    Uses LEFT JOIN to keep all records from adss_data_part1, even if they don’t have a match in beale_decoder.
    Commits the view creation to MySQL.


In [39]:
# function to execute SQL queries
def run_query(query):
    # try: (indent next for error handling)
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
        print(row)


In [36]:
# Function to connect to the database
def connect_to_db(database_name):
    db_config = {
        "host": "mysqldchen.mysql.database.azure.com",
        "user": "dchenAdmin",
        "password": "507password!",
        "database": database_name,
        "port": 3306
    }
    return mysql.connector.connect(**db_config)

add foreign key to adss_data_part1


In [ ]:
import mysql.connector

# Function to connect to the MySQL database
def connect_to_db(database_name):
    return mysql.connector.connect(
        host="mysqldchen.mysql.database.azure.com",
        user="dchenAdmin",
        password="507password!",
        database=database_name,
        port=3306
    )

# Function to execute SQL queries
def run_query(query):
    conn = connect_to_db("icpsr_03088")
    cursor = conn.cursor()
    
    cursor.execute(query)
    conn.commit()

    cursor.close()
    conn.close()

# Run SQL commands separately
conn = connect_to_db("icpsr_03088")
cursor = conn.cursor()

# Ensure beale column is indexed
# cursor.execute("ALTER TABLE adss_data_part1 ADD INDEX idx_beale (beale);")import mysql.connector

# Function to create a view for unique facid and count of cases
def create_facid_case_count_view():
    try:
        # Connect to the MySQL database
        conn = mysql.connector.connect(
            host="mysqldchen.mysql.database.azure.com",
            user="dchenAdmin",
            password="507password!",
            database="icpsr_03088",
            port=3306
        )

        cursor = conn.cursor()
        
        # SQL query to create or replace the view
        query = """
        CREATE OR REPLACE VIEW facid_case_count_view AS
        SELECT 
            facid, 
            COUNT(caseid) AS case_count
        FROM adss_data_part1
        GROUP BY facid;
        """
        
        # Execute the query
        cursor.execute(query)
        conn.commit()

        print("✅ View 'facid_case_count_view' created successfully.")

    except mysql.connector.Error as e:
        print(f"❌ Error: {e}")
    
    finally:
        # Close the cursor and connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Run the function
create_facid_case_count_view()


In [19]:
# SQL Query to create stored procedure to decode 1 as yes and 2 as no
query = """
DELIMITER //

CREATE PROCEDURE decode_yes_no(IN input_value INT, OUT decoded_value VARCHAR(3))
BEGIN
    SET decoded_value = 
        CASE 
            WHEN input_value = 1 THEN 'Yes'
            WHEN input_value = 2 THEN 'No'
            ELSE 'Unknown'
        END;
END //

DELIMITER ;

"""

# Run the query using the existing function
run_query(query)

print("✅ stored procedure 'decode_yes_no' created successfully.")

✅ stored procedure 'decode_yes_no' created successfully.


In [21]:
# call stored procedure to decode yes no values

query = """
SET @decoded_b1b1 = NULL;
CALL decode_yes_no(1, @decoded_b1b1);  -- Replace 1 with actual column value
SELECT @decoded_b1b1;  -- Retrieve the decoded value

"""

# Run the query using the existing function
run_query(query)

print("✅ 'b1b1' labeled successfully.")


✅ View 'adss_data_part1_simplified' created successfully.


In [22]:
#  Use a Stored Function to Decode the 'Column
query = """
DELIMITER //

CREATE FUNCTION decode_yes_no(input_value INT) RETURNS VARCHAR(3)
DETERMINISTIC
BEGIN
    RETURN CASE 
        WHEN input_value = 1 THEN 'Yes'
        WHEN input_value = 2 THEN 'No'
        ELSE 'Unknown'
    END;
END //

DELIMITER ;
"""

# Run the query using the existing function
run_query(query)

print("✅ 'function decode_yes_no' created successfully.")

✅ 'function decode_yes_no' created successfully.


In [25]:
# SQL query to create the view with beale_labeled
query = """
CREATE OR REPLACE VIEW adss_data_decoded AS
SELECT 
    a.caseid, 
    a.facid, 
    a.b1b1 AS original_b1b1,
    decode_yes_no(a.b1b1) AS b1b1_detox_offered_labeled,
    a.b1c1 AS original_b1c1,
    decode_yes_no(a.b1c1) AS b1c1_rehab_offered_labeled,
    b.beale_label AS beale_labeled
FROM adss_data_part1 a
LEFT JOIN beale_decoder b ON a.beale = b.beale_code;
"""

# Run the query using the existing function
run_query(query)

print("✅ View 'adss_data_decoded' created successfully with beale_labeled.")


✅ View 'adss_data_decoded' created successfully with beale_labeled.


ADSS PART3 DRUG CODE LABELING FUNCTION 

In [34]:
# SQL query to create the view with beale_labeled
query = """
DELIMITER //

CREATE FUNCTION decode_substance_p3_a57(input_value INT) RETURNS VARCHAR(50)
DETERMINISTIC
BEGIN
    RETURN CASE 
        WHEN input_value = 0 THEN 'NO SUBSTANCE OF CHOICE'
        WHEN input_value = 40 THEN 'CRACK'
        WHEN input_value = 41 THEN 'COCAINE'
        WHEN input_value = 42 THEN 'HEROIN'
        WHEN input_value = 43 THEN 'METHADONE-NON-TX'
        WHEN input_value = 44 THEN 'OTHER OPIATES'
        WHEN input_value = 45 THEN 'BARBITURATES'
        WHEN input_value = 46 THEN 'BENZODIAZEPINES'
        WHEN input_value = 47 THEN 'OTHER SEDATIVES/HYPNOTICS'
        WHEN input_value = 48 THEN 'AMPHETAMINES'
        WHEN input_value = 49 THEN 'MARIJUANA, HASHISH, THC'
        WHEN input_value = 50 THEN 'HALLUCINOGENS'
        WHEN input_value = 51 THEN 'INHALANTS'
        WHEN input_value = 52 THEN 'OVER-THE-COUNTER'
        WHEN input_value = 53 THEN 'ALCOHOL'
        WHEN input_value = 54 THEN 'TOBACCO'
        WHEN input_value = 55 THEN 'OTHER DRUG1'
        WHEN input_value = 56 THEN 'OTHER DRUG2'
        WHEN input_value = 96 THEN 'MULTIPLE DRUGS NAMED'
        WHEN input_value = -7 THEN 'NOT ASCERTAINED'
        ELSE 'UNKNOWN'
    END;
END //

DELIMITER ;

"""

# Run the query using the existing function
run_query(query)

print("✅ Function 'decode_substance_p3_a57' created successfully .")

✅ Function 'decode_substance_p3_a57' created successfully .


In [41]:
# p3 a73 labeling function REASON FOR DISCHARGE 
query = """
DELIMITER //

CREATE FUNCTION decode_reason_for_discharge(input_value INT) RETURNS VARCHAR(100)
DETERMINISTIC
BEGIN
    RETURN CASE 
        WHEN input_value = 0 THEN 'CLIENT DECEASED'
        WHEN input_value = 1 THEN 'COMPLETED PLANNED TREATMENT'
        WHEN input_value = 2 THEN 'DIDN''T COMPLETE TX, REFERRED/TRANSFERRED'
        WHEN input_value = 3 THEN 'DIDN''T COMPLETE TX, INSURANCE BENEFITS ENDED'
        WHEN input_value = 4 THEN 'DIDN''T COMPLETE TX, NO PAYMENT SOURCE'
        WHEN input_value = 5 THEN 'DIDN''T COMPLETE TX BY ADMINISTRATION CHOICE'
        WHEN input_value = 6 THEN 'DIDN''T COMPLETE TX BY CLIENT CHOICE'
        WHEN input_value = 7 THEN 'DIDN''T COMPLETE TX, INCARCERATED'
        WHEN input_value = 8 THEN 'DIDN''T COMPLETE TX, NOT OTHERWISE SPECIFIED'
        WHEN input_value = 9 THEN 'PENDING COMPLETION OF ASSIGNMENT'
        WHEN input_value = 10 THEN 'MUTUAL AGREEMENT TO TERMINATE'
        WHEN input_value = 11 THEN 'LAWYER TERMINATED THERAPY, COURT TERMINATION'
        WHEN input_value = 12 THEN 'TREATMENT NEVER STARTED'
        WHEN input_value = 13 THEN 'PROBLEMS WITH TRANSPORTATION'
        WHEN input_value = 14 THEN 'EVALUATION ONLY'
        WHEN input_value = 15 THEN 'ARRESTED'
        WHEN input_value = 16 THEN 'MOVED AWAY'
        WHEN input_value = 17 THEN 'DROPOUT, PHYSICAL ILLNESS (E.G., CANCER)'
        WHEN input_value = 18 THEN 'REFERRED TO NON-SA TREATMENT'
        WHEN input_value = 19 THEN 'MEDICALLY DISCHARGE AFTERCARE'
        WHEN input_value = 20 THEN 'HAVING A BABY'
        WHEN input_value = -8 THEN 'INAPPLICABLE'
        WHEN input_value = -9 THEN 'INAPPLICABLE'
        WHEN input_value = -7 THEN 'NOT ASCERTAINED'
        ELSE 'UNKNOWN'
    END;
END //

DELIMITER ;

"""

# Run the query using the existing function
run_query(query)

print("✅ 'function A73 - REASON FOR DISCHARGE' created successfully.")

✅ 'function A73 - REASON FOR DISCHARGE' created successfully.


In [42]:
# p3 a74a labeling function PRIMARY DIAGNOSIS 
query = """DELIMITER //

CREATE FUNCTION decode_primary_diagnosis(input_value INT) RETURNS VARCHAR(100)
DETERMINISTIC
BEGIN
    RETURN CASE 
        WHEN input_value = 0 THEN 'NO DIAGNOSIS'
        WHEN input_value = 1 THEN 'ALCOHOL-INDUCED DISORDER'
        WHEN input_value = 2 THEN 'SUBSTANCE-INDUCED DISORDER'
        WHEN input_value = 3 THEN 'ALCOHOL INTOXICATION'
        WHEN input_value = 4 THEN 'ALCOHOL DEPENDENCE'
        WHEN input_value = 5 THEN 'OPIOID DEPENDENCE'
        WHEN input_value = 6 THEN 'COCAINE DEPENDENCE'
        WHEN input_value = 7 THEN 'CANNABIS DEPENDENCE'
        WHEN input_value = 8 THEN 'OTHER SUBSTANCE DEPENDENCE'
        WHEN input_value = 9 THEN 'ALCOHOL ABUSE'
        WHEN input_value = 10 THEN 'CANNABIS ABUSE'
        WHEN input_value = 11 THEN 'OTHER SUBSTANCE ABUSE'
        WHEN input_value = 12 THEN 'OPIOID ABUSE'
        WHEN input_value = 13 THEN 'COCAINE ABUSE'
        WHEN input_value = 14 THEN 'ANXIETY DISORDERS'
        WHEN input_value = 15 THEN 'DEPRESSIVE DISORDERS'
        WHEN input_value = 16 THEN 'SCHIZOPHRENIA/OTHER PSYCHOTIC DISORDERS'
        WHEN input_value = 17 THEN 'BIPOLAR DISORDERS'
        WHEN input_value = 18 THEN 'ATTENTION DEFICIT/DISRUPTIVE BEHAVIOR DISORDER'
        WHEN input_value = 19 THEN 'OTHER MENTAL HEALTH CONDITION'
        WHEN input_value = 20 THEN 'OTHER CONDITION'
        WHEN input_value = -9 THEN 'NOT APPLICABLE'
        ELSE 'UNKNOWN'
    END;
END //

DELIMITER ;
"""

# Run the query using the existing function
run_query(query)

print("✅ 'function A74A - PRIMARY DIAGNOSIS' created successfully.")

✅ 'function A74A - PRIMARY DIAGNOSIS' created successfully.
